# Process Data

The purpose of this `.ipynb` is to process `imu.csv` data to get `pitch`, and combine this with `gaze_positions.csv` to get `elevation`, and combine `pitch` with `elevation` to get `gaze angle`. 

## Steps
1. Establish target directory (under `data/`, select folder for session: `session_XX` where XX is a two-digit number).
2. Load `imu.csv` and `gaze_positions.csv` into dataframes.
3. Combine `pitch` with `elevation` to get `gaze angle`.
4. Save processed data to new CSV file (time step + gaze angle), as `gaze_angle.csv`.


In [ ]:
import pandas as pd
import plotly.express as px

# if we want to save the output file here from "Run All", make SAVE_OUTPUT = True
SAVE_OUTPUT = False

In [67]:
# 1: Establish target directory 

SESSION_ID = 'session_02'

target_dir = 'data/' + SESSION_ID

if target_dir:
    print(target_dir)
else: 
    print('Target directory doesn not exist')

data/session_02


In [68]:
# 2. Load in csv's

# Load IMU data
imu_df = pd.read_csv(f'{target_dir}/imu.csv')
# Load gaze positions data
gaze_df = pd.read_csv(f'{target_dir}/gaze_positions.csv')

print("IMU data head:")
print(imu_df.head())
print("\nGaze positions data head:")
print(gaze_df.head())

IMU data head:
        timestamp [ns]  gyro x [deg/s]  gyro y [deg/s]  gyro z [deg/s]  \
0  1760637455382874689       -0.427246       -4.924774       16.378403   
1  1760637455390873689        1.401901       -5.842209       16.439438   
2  1760637455398872689        5.125046       -6.147385       16.439438   
3  1760637455406871689        9.639740       -5.596161       16.683578   
4  1760637455417618689       16.475677       -5.962372       16.927720   

   acceleration x [G]  acceleration y [G]  acceleration z [G]  roll [deg]  \
0            0.100098           -0.451660            0.367676   -5.264761   
1            0.088379           -0.447754            0.307617   -5.120203   
2            0.091797           -0.437988            0.243652   -4.970989   
3            0.080566           -0.411621            0.180176   -4.810644   
4            0.090820           -0.415039            0.214844   -4.653854   

   pitch [deg]  yaw [deg]  quaternion w  quaternion x  quaternion y  \
0   -5

In [69]:
# Downsample gaze data to IMU frame rate: for each IMU frame, find the nearest gaze elevation
imu_df_sorted = imu_df.sort_values('timestamp [ns]')
gaze_df_sorted = gaze_df.sort_values('timestamp [ns]')
merged_df = pd.merge_asof(
    imu_df_sorted,
    gaze_df_sorted[['timestamp [ns]', 'elevation [deg]']],
    on='timestamp [ns]',
    direction='nearest'
    # Optionally, set a tolerance to restrict matching distance
    # tolerance=10000  # e.g., 10,000 ns = 0.01 ms
)

# If no gaze frame is close enough, elevation will be NaN
# merged_df = merged_df.dropna(subset=['elevation [deg]'])
# merged_df = merged_df.fillna({'elevation [deg]': 0})
# Uncomment above as needed

# Now, merged_df has IMU frames with nearest gaze elevation
# Calculate gaze angle
merged_df['gaze angle [deg]'] = merged_df['pitch [deg]'] + merged_df['elevation [deg]']
gaze_angle_df = merged_df[['timestamp [ns]', 'gaze angle [deg]']].copy()
# Include pitch and elevation in final df for saving
gaze_angle_df.loc[:, 'pitch [deg]'] = merged_df['pitch [deg]']
gaze_angle_df.loc[:, 'elevation [deg]'] = merged_df['elevation [deg]']
print(f"Size of dataframe: {len(merged_df)}")
print(gaze_angle_df.head())

Size of dataframe: 9191
        timestamp [ns]  gaze angle [deg]  pitch [deg]  elevation [deg]
0  1760637455382874689        -71.965136   -53.220745       -18.744391
1  1760637455390873689        -71.958933   -53.214542       -18.744391
2  1760637455398872689        -71.917193   -53.172802       -18.744391
3  1760637455406871689        -71.831324   -53.086933       -18.744391
4  1760637455417618689        -71.691737   -52.947346       -18.744391


In [ ]:
# Calculate time in seconds, first frame is 0
gaze_angle_df = gaze_angle_df.copy()
start_ns = gaze_angle_df['timestamp [ns]'].iloc[0]
gaze_angle_df['time_sec'] = (gaze_angle_df['timestamp [ns]'] - start_ns) / 1e9

fig = px.scatter(
    gaze_angle_df,
    x='time_sec',
    y='gaze angle [deg]',
    title='Gaze Angle Over Time',
    labels={'gaze angle [deg]': 'Gaze Angle (deg)', 'time_sec': 'Time (sec)'},
    render_mode='webgl'
)
fig.update_traces(mode='lines+markers')
fig.update_yaxes(range=[-90, 45])
fig.show()

In [ ]:
from ipywidgets import widgets
from IPython.display import display

gaze_angle_df.head()
import plotly.graph_objects as go

def plot_gaze_metric(metric):
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=gaze_angle_df['time_sec'],
        y=gaze_angle_df[metric],
        mode='lines+markers',
        name=metric,
        line=dict(width=2),
        marker=dict(size=4)
    ))
    fig2.update_layout(
        title=f'{metric} Over Time',
        xaxis_title='Time (sec)',
        yaxis_title=f'{metric}',
        yaxis=dict(range=[-90, 45] if metric in ['gaze angle [deg]', 'elevation [deg]', 'pitch [deg]'] else None)
    )
    fig2.show()

dropdown = widgets.Dropdown(
    options=['elevation [deg]', 'pitch [deg]'],
    value='elevation [deg]',
    description='Metric:',
    disabled=False,
)

widgets.interact(plot_gaze_metric, metric=dropdown)

interactive(children=(Dropdown(description='Metric:', options=('elevation [deg]', 'pitch [deg]'), value='eleva…

<function __main__.plot_gaze_metric(metric)>

In [72]:
# Save entire gaze_angle_df to CSV in the target directory

if SAVE_OUTPUT == True:
    out_path = f'{target_dir}/gaze_angle.csv'
    gaze_angle_df.to_csv(out_path, index=False)
    print(f"Saved {len(gaze_angle_df)} rows to {out_path}")
else: 
    print("No output is being saved")


No output is being saved
